# Machine Learning

## Pylab 4

Linear Discriminant Analysis e Quadratic Discriminant Analysis.

Prof. Neylson Crepalde

Instituto Metodista Izabela Hendrix

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, auc

# ML Algorithms
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Vamos ler o banco de dados **Smarket** disponível no pacote ISLR para linguagem R. Segundo os autores,

"This data set consists of percentage returns for the S&P 500 stock index over 1,250 days, from the beginning of 2001 until the end of 2005. For each date, we have recorded the percentage returns for each of the five previous trading days, Lag1 through Lag5. We have also recorded Volume (the number of shares traded on the previous day, in billions), Today (the percentage return on the date in question) and Direction(whether the market was Up or Down on this date)."

In [2]:
# Importando o banco de dados
bd = pd.read_csv("https://raw.githubusercontent.com/neylsoncrepalde/ML_classes/master/Data/Smarket.csv")
bd.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [3]:
# Olhando as variáveis
bd.describe()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.00000,1250.000000,1250.000000
mean,2003.016000,0.003834,0.003919,0.001716,0.001636,0.00561,1.478305,0.003138
std,1.409018,1.136299,1.136280,1.138703,1.138774,1.14755,0.360357,1.136334
min,2001.000000,-4.922000,-4.922000,-4.922000,-4.922000,-4.92200,0.356070,-4.922000
25%,2002.000000,-0.639500,-0.639500,-0.640000,-0.640000,-0.64000,1.257400,-0.639500
50%,2003.000000,0.039000,0.039000,0.038500,0.038500,0.03850,1.422950,0.038500
75%,2004.000000,0.596750,0.596750,0.596750,0.596750,0.59700,1.641675,0.596750
max,2005.000000,5.733000,5.733000,5.733000,5.733000,5.73300,3.152470,5.733000


In [5]:
bd.Direction.value_counts()

Up      648
Down    602
Name: Direction, dtype: int64

In [7]:
# Matriz de correlação
bd.drop(columns="Direction").corr()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
Year,1.000000,0.029700,0.030596,0.033195,0.035689,0.029788,0.539006,0.030095
Lag1,0.029700,1.000000,-0.026294,-0.010803,-0.002986,-0.005675,0.040910,-0.026155
Lag2,0.030596,-0.026294,1.000000,-0.025897,-0.010854,-0.003558,-0.043383,-0.010250
Lag3,0.033195,-0.010803,-0.025897,1.000000,-0.024051,-0.018808,-0.041824,-0.002448
Lag4,0.035689,-0.002986,-0.010854,-0.024051,1.000000,-0.027084,-0.048414,-0.006900
Lag5,0.029788,-0.005675,-0.003558,-0.018808,-0.027084,1.000000,-0.022002,-0.034860
Volume,0.539006,0.040910,-0.043383,-0.041824,-0.048414,-0.022002,1.000000,0.014592
Today,0.030095,-0.026155,-0.010250,-0.002448,-0.006900,-0.034860,0.014592,1.000000


No geral, as correlações são baixas. A única correlação mais expressiva é entre Year e Volume (0.539)

Vamos usar a análise discriminante para prever a direção no mercado dados Lag1 e Lag2

In [8]:
bd1 = pd.get_dummies(bd, drop_first=True)
print(bd1.columns)

Index(['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Today',
       'Direction_Up'],
      dtype='object')


In [0]:
X = bd1[["Lag1", "Lag2"]]
y = bd1.Direction_Up
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

Vamos usar o algoritmo de LDA do SKLEARN. Ele possui 3 algortimos. É curioso notar que o algortimo default `svd` **não leva em conta a matriz de covariância** e, portanto é mais adequada a dados com mais features. O algoritmo `eigen` leva em conta a matriz de covariância e está mais alinhado com o método descrito no livro-texto. Experimentemos os dois métodos.

In [0]:
lda1 = LinearDiscriminantAnalysis(solver="svd")

In [17]:
lda1.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [19]:
# Checking the results
yhat = lda1.predict(X_test)
auc = roc_auc_score(y_test, yhat)
print("AUC: ", auc)

AUC:  0.5080356126976903


Horrível!!!! Vamos tentar adicionar mais variáveis nesse modelo.

In [0]:
X = bd1[["Lag1", "Lag2", "Lag3", "Lag4", "Lag5", "Year", "Volume"]]
y = bd1.Direction_Up
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [21]:
lda2 = LinearDiscriminantAnalysis(solver="svd")
lda2.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [22]:
yhat = lda2.predict(X_test)
auc = roc_auc_score(y_test, yhat)
print("AUC: ", auc)

AUC:  0.5193850267379679


Não foi muito melhor. Vamos tentar com outro algoritmo.

In [23]:
lda3 = LinearDiscriminantAnalysis(solver="eigen")
lda3.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='eigen', store_covariance=False, tol=0.0001)

In [24]:
yhat = lda3.predict(X_test)
auc = roc_auc_score(y_test, yhat)
print("AUC: ", auc)

AUC:  0.5354562521333486


Melhorou um pouco. Mesmo asim, esse AUC está abaixo do aceitável. Vamos tentar com QDA

In [25]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)

In [26]:
yhat = qda.predict(X_test)
auc = roc_auc_score(y_test, yhat)
print("AUC: ", auc)

AUC:  0.5063431562179997


Vamos tentar aplicar esse método para os dados do banco 

In [27]:
bd = pd.read_csv("https://raw.githubusercontent.com/neylsoncrepalde/ML_classes/master/Data/Default.csv")
bd.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [0]:
bd1 = pd.get_dummies(bd, drop_first=True)
X = bd1[["student_Yes", "balance", "income"]]
y = bd1.default_Yes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [29]:
lda1 = LinearDiscriminantAnalysis(solver="eigen")
lda1.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='eigen', store_covariance=False, tol=0.0001)

In [31]:
yhat = lda1.predict(X_test)
auc = roc_auc_score(y_test, yhat)
print("AUC: ", auc)

AUC:  0.8646262640762469


Wow!!! Nesses dados a LDA teve um resultado muito mais interessante! Vamos ver se QDA pode superar isso:

In [32]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)

In [33]:
yhat = qda.predict(X_test)
auc = roc_auc_score(y_test, yhat)
print("AUC: ", auc)

AUC:  0.6574499752567817


Não foi tão bom. Imaginamos que a linha de decisão linear foi melhor do que a quadrática dado que o balance faz uma separação muito estanque entre as categorias de Default.